In [1]:
# !pip install catboost==0.15.2
# !pip install tqdm
# !pip install xlrd
import numpy as np
import pandas as pd
import catboost as cbt
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss
import gc
import math
import time
from tqdm import tqdm
import datetime
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
train = pd.read_csv('../data/first_round_training_data.csv')
test = pd.read_csv('../data/first_round_testing_data.csv')
submit = pd.read_csv('../data/submit_example.csv')
data = train.append(test).reset_index(drop=True)
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
data['label'] = data['Quality_label'].map(dit)

In [12]:
fea_list = [i for i in test.columns if i not in ['Group']] 
lbl = LabelEncoder()

In [39]:
feature_name = [i for i in data.columns if i not in ['Attribute1','Attribute10', 'Attribute2', 'Attribute3', 'Attribute4', 'Attribute5',
 'Attribute6', 'Attribute7', 'Attribute8', 'Attribute9', 'Quality_label','Group','label']]
tr_index = ~data['label'].isnull()  #isnull()函数，会将null值返回true, 非null值返回false; ~符号取反
X_train = data[tr_index][feature_name].reset_index(drop=True).astype(str)
y = data[tr_index]['label'].reset_index(drop=True).astype(int)
X_test = data[~tr_index][feature_name].reset_index(drop=True)

print(X_train.shape,X_test.shape)
seed = 2019

oof = np.zeros((X_train.shape[0],4))
prediction=np.zeros((X_test.shape[0],4))
skf = StratifiedKFold(n_splits=5, random_state=seed, shuffle=True)
for index, (train_index, test_index) in enumerate(skf.split(X_train, y)):
    print(index)
    train_x, test_x, train_y, test_y = X_train.iloc[train_index], X_train.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    gc.collect()
    print(train_x.shape)
    cbt_model = cbt.CatBoostClassifier(iterations=100000,learning_rate=0.01,verbose=300,max_depth=7,#eval_metric='Accuracy',
                                    early_stopping_rounds=500,task_type='GPU',devices='0',cat_features=fea_list,
                                    loss_function='MultiClass')
    cbt_model.fit(train_x, train_y ,eval_set=(test_x,test_y))
    oof[test_index] += cbt_model.predict_proba(test_x)
    prediction += cbt_model.predict_proba(X_test)/5
    gc.collect()
    
print('logloss',log_loss(pd.get_dummies(y).values, oof))
print('ac',accuracy_score(y, np.argmax(oof,axis=1)))

(6000, 10) (6000, 10)
0
(4798, 10)
0:	learn: -1.3815709	test: -1.3813737	best: -1.3813737 (0)	total: 12.9ms	remaining: 21m 26s
300:	learn: -1.0775586	test: -1.1379283	best: -1.1379283 (300)	total: 8.42s	remaining: 46m 29s
600:	learn: -1.0237601	test: -1.1298449	best: -1.1298449 (600)	total: 12.4s	remaining: 34m 6s
900:	learn: -0.9814390	test: -1.1289773	best: -1.1287914 (867)	total: 18.7s	remaining: 34m 12s
1200:	learn: -0.9423287	test: -1.1294189	best: -1.1281514 (1039)	total: 25.5s	remaining: 35m
1500:	learn: -0.9077797	test: -1.1311549	best: -1.1281514 (1039)	total: 32.7s	remaining: 35m 43s
bestTest = -1.128151384
bestIteration = 1039
Shrink model to first 1040 iterations.


CatBoostError: Invalid type for cat_feature[0,1]=0.1956804776694072 : cat_features must be integer or string, real number values and NaN values should be converted to string.

In [ ]:
sub = test[['Group']]
prob_cols = [i for i in submit.columns if i not in ['Group']]
for i, f in enumerate(prob_cols):
    sub[f] = prediction[:, i]
for i in prob_cols:
    sub[i] = sub.groupby('Group')[i].transform('mean')
sub = sub.drop_duplicates()
sub.to_csv("submission.csv",index=False)

In [37]:
X_train = data[tr_index][feature_name].reset_index(drop=True).apply(str)
X_train[0:1]

Parameter1    0            0.001660\n1            1.601749\n...
dtype: object

In [41]:
fea_list[0,1]

TypeError: list indices must be integers or slices, not tuple